# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
pip install gmaps

  Created wheel for gmaps: filename=gmaps-0.9.0-py2.py3-none-any.whl size=2076111 sha256=1d922baedf9f818d4002dd64ac59fb23aa934ad962e29672fa58a8b0d8405012
  Stored in directory: c:\users\farah\appdata\local\pip\cache\wheels\66\ab\29\3032938612273665f0fa4d2fab4e3ecfe86480eaaa40aaee91
Successfully built gmaps


In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [19]:
weather_csv_file = "./city_weather_data.csv"

weather_df = pd.read_csv(weather_csv_file)

weather_df.head()

,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,busselton,-33.6500,115.3333,56.07,68.0,100.0,8.77,AU,1.665319e+09
1,rikitea,-23.1203,-134.9692,71.31,66.0,10.0,17.13,PF,1.665319e+09
2,yellowknife,62.4560,-114.3525,41.56,100.0,0.0,8.05,CA,1.665319e+09
3,cape town,-33.9258,18.4232,78.12,53.0,0.0,25.32,ZA,1.665319e+09
4,hasaki,35.7333,140.8333,63.00,83.0,100.0,3.76,JP,1.665320e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [24]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# Also, handle NaN values
weather_df = weather_df.dropna()
humidity = weather_df["Humidity"].astype(float)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = weather_df[["Lat", "lng"]].astype(float)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(weather_df["Humidity"]),
                                 point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
# Narrow down the DataFrame to find your ideal weather condition.
filtered_weather_df = weather_df

# Drop any rows that don't contain all three conditions. Want to be sure the weather is ideal.

# A max temperature lower than 80 degrees but higher than 70.
filtered_weather_df = filtered_weather_df.loc[(filtered_weather_df["Max Temp"] < 80) & (filtered_weather_df["Max Temp"] > 70)]

# Wind speed less than 10 mph.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Wind Speed"] < 10]

# Zero cloudiness.
filtered_weather_df = filtered_weather_df.loc[filtered_weather_df["Cloudiness"] == 0]

# Drop any rows with null values
filtered_weather_df = filtered_weather_df.dropna()

filtered_weather_df

,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
101,kapaa,22.0752,-159.3190,73.38,83.0,0.0,9.22,US,1.665319e+09
236,kahului,20.8947,-156.4700,75.85,78.0,0.0,1.99,US,1.665320e+09
263,castro,-24.7911,-50.0119,71.38,55.0,0.0,4.09,BR,1.665320e+09
308,cabo san lucas,22.8909,-109.9124,72.61,69.0,0.0,5.59,MX,1.665320e+09
435,henderson,36.0397,-114.9819,73.24,40.0,0.0,5.75,US,1.665320e+09
455,mananjary,-21.2167,48.3333,76.60,53.0,0.0,9.55,MG,1.665320e+09
529,aras,42.5617,-2.3560,72.75,56.0,0.0,5.75,ES,1.665320e+09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
hotel_df = filtered_weather_df

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    # print(json.dumps(response, indent=4, sort_keys=True))
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 101: kapaa.
Missing field/result... skipping.
------------
Retrieving Results for Index 236: kahului.
Missing field/result... skipping.
------------
Retrieving Results for Index 263: castro.
Missing field/result... skipping.
------------
Retrieving Results for Index 308: cabo san lucas.
Missing field/result... skipping.
------------
Retrieving Results for Index 435: henderson.
Missing field/result... skipping.
------------
Retrieving Results for Index 455: mananjary.
Missing field/result... skipping.
------------
Retrieving Results for Index 529: aras.
Missing field/result... skipping.
------------


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [34]:
hotel_df

,City,Lat,lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
101,kapaa,22.0752,-159.3190,73.38,83.0,0.0,9.22,US,1.665319e+09
236,kahului,20.8947,-156.4700,75.85,78.0,0.0,1.99,US,1.665320e+09
263,castro,-24.7911,-50.0119,71.38,55.0,0.0,4.09,BR,1.665320e+09
308,cabo san lucas,22.8909,-109.9124,72.61,69.0,0.0,5.59,MX,1.665320e+09
435,henderson,36.0397,-114.9819,73.24,40.0,0.0,5.75,US,1.665320e+09
455,mananjary,-21.2167,48.3333,76.60,53.0,0.0,9.55,MG,1.665320e+09
529,aras,42.5617,-2.3560,72.75,56.0,0.0,5.75,ES,1.665320e+09


In [32]:
# Assign the marker layer to a variable
locations = hotel_df[["Lat", "lng"]]
markers = gmaps.marker_layer(locations)

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig

KeyError: 'Hotel Name'